In [2]:
## fasta files used for this analysis are whole genome assemblies from 
## for custom db they need to be concatinated into one fasta file
## dowload fasta files and concatenate into one file
## examples
## for our test simulation I am using C.costata, D.mel, S.sech, D.sim, D.maur ( sech, sim, mar = close to mel ,cos = farthest)
! mkdir -p sequences
! cat sequences/D.mauritiana.fasta sequences/D.melanogaster.fasta sequences/D.sechellia.fasta sequences/D.simulans.fasta sequences/C.costata.fasta > sequences/combined_genomes.fasta 

In [ ]:
## make db will throw an error if any duplicate ids so we check for this
## some fasta files may still have duplicates and this will cause an error in make db, therefore we need to remove duplicates fasta files
## this can be done in 2 steps
## step 1) evaluate sequences/test_genomes_modified.fasta for duplicates and count if there are duplicates. 
## grep searches for > (start of fasta id line), uniq d prints out any duplicated non unique entries, wc -l counts the lines
! cat sequences/combined_genomes.fasta | grep '^>' | sort | uniq -d | wc -l 

In [4]:
## make db is particular about format of id so here we need to process the id's from our data to be compatable with blast makedb
## to parse ids for make db need 1) unique entries 2) names to not have blank spaces and other characters/less than 50 long
## for these files, we are keeping all alpha numberics and removing the end isolate number and linking names with "-"
! sed 's/[^a-zA-Z0-9>_]/-/g; s/---isolate.*//' sequences/combined_genomes.fasta > sequences/combined_genomes_modified_ids.fasta

In [10]:
## total sequences. this should be the number of sequences in output database. 
! cat sequences/combined_genomes_modified_ids.fasta | grep ">" | wc -l
! head sequences/combined_genomes_modified_ids.fasta

    2367
>contig_1--organism-Drosophila-mauritiana
gattccgagacagaaaagtcaacatgagccatacgctcacctggtctatactgaatatcaaattcgaaagattgcatgta
cgaccaccactctatgaactctaaaggtcagttctatctttgtgggatgaggctttaagcgaattgcaatcagtaaacac
aacaaaatgacggccatgcaagtaatggcgaaagtgttttatagaattcggtatacgcgattctgctgaagatgtcatct
gctaaaatatttcgataactcgcggcttctgttcaatccggtgcagcgaaaatagtgccgtagccgacagaatcaaggcg
tctgtgtggagcttcgatagaaaattgtgggtcgaatatagtaaggggacaggttcgttagttaaaatagatattacttt
ttgccgtatttgttcatgctcaatttcacaaaaatgacccgctctttgaagtaaaggcgtaaagagggtttcacttcagt
cagaaaacttgggtacaaattgtcgaaaatacgagcatcagctccaataaacgtctcaacagaaagtgacggactgggtg
gaggtggcaaattcgaaagggtcgaattttcctgacggccgaatctcaccttgacttaactcgaaacccaaatattcaac
acaagacttgagaaaggaacattttgtaatattaaaagaaaatccagcatgcgtaagtgcttttcttgcagtaattactc


In [5]:
# make database of sequences from concatinated sequence files (currently having issue bc won't parse ids correctly...)
# I removed the -parse_seqids and it made a list of files with db base name (need unique files for this )
# we will need to figure out why this is not working and add parse ids later 
! mkdir -p database
! makeblastdb -dbtype nucl -in sequences/combined_genomes_modified_ids.fasta  -out database/genomes_db -blastdb_version 5 -parse_seqids



Building a new DB, current time: 01/23/2021 15:44:38
New DB name:   /Users/giana.cirolia/enhancer_analysis_codebase/database/genomes_db
New DB title:  sequences/combined_genomes_modified_ids.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 2367 sequences in 11.4137 seconds.




In [6]:
# check that your db created a database 
! blastdbcmd -db database/genomes_db -info

Database: sequences/combined_genomes_modified_ids.fasta
	2,367 sequences; 871,498,291 total bases

Date: Jan 23, 2021  3:44 PM	Longest sequence: 35,111,255 bases

BLASTDB Version: 5

Volumes:
	/Users/giana.cirolia/enhancer_analysis_codebase/database/genomes_db


In [9]:
# count the number of sequence ids in your data base 
! blastdbcmd -db database/genomes_db -entry all -outfmt %i | wc -l

    2367


In [ ]:
# if sequence ids equals input fastas you are correct. Here our sequence ids are not the names we want but its a start.